In [3]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import numpy as np

- Loading `diabetes` dataset

In [2]:
diabetes: np.ndarray = load_diabetes()

- Splitting data into training set and test set

In [4]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, random_state=55)